# Introduction to Data Science
## Homework 3

Student Name: Test!

Student Netid: 
***

### Part 1 (5 Points)
Assume that $X$ and $Y$ are discrete random variables. The formulas for Mutual Information, Entropy and Conditional Entropy are given by:

$$\text{Mutual Information} = \sum_{y \in Y} \sum_{x \in X} p(x, y) \cdot log\frac{p(x, y)}{p(x)p(y)}$$

$$\text{Entropy} = H(Y) = -\sum_{y \in Y} p(y) \cdot log(p(y))$$

$$\text{Conditional Entropy} = H(Y \mid X) = \sum_{x \in X} p(x) \cdot H(Y \mid X = x)$$

Show mathematically that $\text{Mutual Information} = \text{Information Gain}$, where $\text{Information Gain} = H(Y) – H(Y \mid X)$. Give the derivation below (note, this can be done using Latek math notation, which renders nicely. See above. Feel free to do it by hand and submit an image of your proof).

Place your answer here!

We will show that $\text{Mutual Information} + H(Y \mid X) = H(Y)$

$$ H(Y \mid X) = \sum_{x \in X} p(x) \cdot H(Y \mid X = x) = 
- \sum_{x \in X} p(x) \sum_{y \in Y} p(y \mid x) \cdot log(p(y \mid x)) \\
\text{ by definition}$$

$$ = - \sum_{x \in X} \sum_{y \in Y} p(x, y) \cdot log(p(y \mid x)) 
= - \sum_{x \in X} \sum_{y \in Y} p(x, y) \cdot log(\frac{p(x,y)}{p(x)}) \\
\text{ since } p(x) \cdot p(y \mid x) = p(x,y)$$

$$ = \sum_{x \in X} \sum_{y \in Y} p(x, y) \cdot log(\frac{p(x)}{p(x,y)}) 
= \sum_{y \in Y} \sum_{x \in X} p(x, y) \cdot log(\frac{p(x)}{p(x,y)})\\
\text{by flipping the log and the sum order}$$

$$ \text{then} \\
\text{Mutual Information} + H(Y \mid X) =
\sum_{y \in Y} \sum_{x \in X} p(x, y) \cdot log\frac{p(x, y)}{p(x)p(y)} + 
\sum_{y \in Y} \sum_{x \in X} p(x, y) \cdot log(\frac{p(x)}{p(x,y)}) 
$$

$$ = \sum_{y \in Y} \sum_{x \in X} p(x, y) \cdot
log\frac{p(x, y) \cdot p(x)}{p(x)p(y) \cdot p(x,y)} $$

$$ = \sum_{y \in Y} \sum_{x \in X} p(x, y) \cdot log\frac{1}{p(y)}
= - \sum_{y \in Y} \sum_{x \in X} p(x, y) \cdot log(p(y))$$

$$ = - \sum_{y \in Y} log(p(y)) \sum_{x \in X} p(x) p(y \mid x) \\
\text{since } p(x,y) = p(y \mid x) \cdot p(x)
$$

$$ = -\sum_{y \in Y} p(y) \cdot log(p(y)) = H(Y) \\
\text{since } \sum_{x \in X} p(x) p(y \mid x) = p(y) \\
\text{QED}
$$

### Part 2 - Preparing a Training Set and Training a Decision Tree (10 Points)
This is a hands-on task where we build a predictive model using Decision Trees discussed in class. For this part, we will be using the data in `cell2cell_data.csv`.

These historical data consist of 39,859 customers: 19,901 customers that churned (i.e., left the company) and 19,958 that did not churn (see the `"churndep"` variable). Here are the data set's 11 possible predictor variables for churning behavior: 

```
Pos.  Var. Name  Var. Description
----- ---------- --------------------------------------------------------------
1     revenue    Mean monthly revenue in dollars
2     outcalls   Mean number of outbound voice calls
3     incalls    Mean number of inbound voice calls
4     months     Months in Service
5     eqpdays    Number of days the customer has had his/her current equipment
6     webcap     Handset is web capable
7     marryyes   Married (1=Yes; 0=No)
8     travel     Has traveled to non-US country (1=Yes; 0=No)
9     pcown      Owns a personal computer (1=Yes; 0=No)
10    creditcd   Possesses a credit card (1=Yes; 0=No)
11    retcalls   Number of calls previously made to retention team
```

The 12th column, the dependent variable `"churndep"`, equals 1 if the customer churned, and 0 otherwise. 

1\. Load the data and prepare it for modeling. Note that the features are already processed for you, so the only thing needed here is split the data into training and testing. Use pandas to create two data frames: train_df and test_df, where train_df has 80% of the data chosen uniformly at random without replacement (test_df should have the other 20%). Also, make sure to write your own code to do the splits. You may use any random() function numpy but DO NOT use the data splitting functions from Sklearn.

In [16]:
import pandas as pd
import numpy as np

In [21]:
# we copied the above description to a meta.txt file
# and can use utilities to extract the column headers
!cat meta.txt | tr -s ' ' | cut -d ' ' -f2 > columns.csv
cols_df = pd.read_csv('columns.csv', header=None)
cols = np.append(cols_df[0][2:].values, 'churndep')
print(cols)
print(len(cols))

['revenue' 'outcalls' 'incalls' 'months' 'eqpdays' 'webcap' 'marryyes'
 'travel' 'pcown' 'creditcd' 'retcalls' 'churndep']
12


In [22]:
# read the file into the dataframe
df = pd.read_csv("cell2cell_data.csv", header=None, names=cols)
display(df.head())
print(df.shape)

,revenue,outcalls,incalls,months,eqpdays,webcap,marryyes,travel,pcown,creditcd,retcalls,churndep
0,48.82,10.00,3.0,26,780,0,0,0,0,1,4,1
1,83.53,20.00,1.0,31,745,1,0,0,0,0,4,1
2,29.99,0.00,0.0,52,1441,0,0,0,1,1,3,1
3,51.42,0.00,0.0,36,59,1,0,0,0,0,4,1
4,37.75,2.67,0.0,25,572,0,0,0,1,1,3,1


(39859, 12)


In [31]:
pct_train = 0.8  # parameter
num_train = round(df.shape[0] * pct_train)
num_test = df.shape[0] - num_train

test_idx = np.random.choice(np.arange(0,df.shape[0]), num_text)
assert num_text == len(test_idx)  # makes sure we have the same number of test indices

In [36]:
test_df = df.iloc[test_idx]
train_df = df[~df.index.isin(test_df.index)]
print(len(train_df), len(test_df))

32637 7972


In [41]:
# let's make sure we have no data overlap
assert len(train_df[[]].join(test_df[[]], how='inner')) == 0

#### Great!, can now continue to the next step

2\. If we had to, how would we prove to ourselves or a colleague that our data was indeed randomly sampled on X? And by prove, I mean empirically, not just showing this person our code. Don't actually do the work, just describe in your own words a test you could here. Hint: think about this in terms of selection bias and use notes from our 2nd lecture.

First we can study the sampled distributions. Between the sampled sets, the statistics (mean, std, qqplot) of each should be similar.

In this particular implementation, since we used np's random to sample and we don't know if the random function is biased in anyway, we can also make sure that the indices of the sampled dataset are distributed uniformly in the sample space for both sets.

3\. Now build and train a decision tree classifier using `DecisionTreeClassifier()` [(manual page)](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) on train_df to predict the `"churndep"` target variable. Make sure to use `criterion='entropy'` when instantiating an instance of `DecisionTreeClassifier()`. For all other settings you should use all of the default options.

In [ ]:
import sklearn

# Code here

4\. Using the resulting model from 2.3, show a bar plot of feature names and their feature importance (hint: check the attributes of the `DecisionTreeClassifier()` object directly in IPython or check the manual!). Make sure the bar plot is sorted by increasing feature importance values.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Code here

5\. Is the relationship between the top 3 most important features (as measured here) negative or positive? If your marketing director asked you to explain the top 3 drivers of churn, how would you interpret the relationship between these 3 features and the churn outcome? What "real-life" connection can you draw between each variable and churn? Make sure to state your answer, and not just show code.

In [ ]:
# Code/answer here

6\. Using the classifier built in 2.3, try predicting "churndep" on both the train_df and test_df data sets. What is the accuracy on each? What is your explanation on the difference (or lackthereof) between the two accuracies?

In [ ]:
# Code here

### Part 3 - Finding a Good Decision Tree (10 Points)
The default options for your decision tree may not be optimal. We need to analyze whether tuning the parameters can improve the accuracy of the classifier.  For the following options `min_samples_split` and `min_samples_leaf`:

1\. Generate a list of 10 values of each for the parameters mim_samples_split and min_samples_leaf. 

In [ ]:
# Code here

min_samples_split_values = None
min_samples_leaf_values = None

2\. Explain in words your reasoning for choosing the above ranges.

3\. For each combination of values in 3.1 (there should be 100), build a new classifier and check the classifier's accuracy on the test data. Plot the test set accuracy for these options. Use the values of `min_samples_split` as the x-axis and generate a new series (line) for each of `min_samples_leaf`.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Code here

4\. Which configuration returns the best accuracy? What is this accuracy? (Note, if you don't see much variation in the test set accuracy across values of min_samples_split or min_samples_leaf, try redoing the above steps with a different range of values), and reassess your answer in Q3.2.

In [ ]:
# Code here

5\. If you were working for a marketing department, how would you use your churn production model in a real business environment? Explain why churn prediction might be good for the business and how one might improve churn by using this model.

Answer here!